<div class="alert alert-block alert-warning">
<b>Disclaimer:</b> The main objective of this jupyter notebook is to show how to fit simple data by
    
- defining a fitting model
- creating the reference data to which the model will be fitted to. In standard cases, this step should be replaced by loading your experimental data
- setting and running the fit   
- extracting and displaying information about the results

The syntax to fit data is minimizer-dependent. Here we focus on a fitting routing provided by the `scipy` python package: <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html">scipy.optimize.curve_fit</a>.   
    
Please also note that the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.
</div>


<a id='Table of Contents'></a><h1>Table of Contents</h1>

- <a href='#def_function'>Definition of the fitting model</a>
- <a href='#imports'>Importing libraries</a>
- <a href='#anim_plot'>Plot of the fitting model</a>
- <a href='#ref_data'>Creating reference data</a>
- <a href='#fitting'>Setting and fitting</a>
- <a href='#plot'>Plotting the results</a>  

(<a href='#Table of Contents'>Top</a>)<a id='imports'></a><h2>Importing libraries</h2>

In [ ]:
# import python modules for plotting, fitting
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# for interactive plot
import ipywidgets

(<a href='#Table of Contents'>Top</a>)<a id='def_function'></a><h2>Definition of the fitting model</h2>

Here we define a simple lorentzian function

In [ ]:
def lorentzian(xx, scale=1.0, center=1.0, hwhm=3.0):
    if hwhm == 0:
        raise ValueError('hwhm of the lorentzian is equal to zero.')
    return scale * hwhm / ((xx-center)**2 + hwhm**2) / np.pi

(<a href='#Table of Contents'>Top</a>)<a id='anim_plot'></a><h2>Plot of the fitting model</h2>

The widget below shows how the variations of the lorentzian's parameters, *Scale*, *Center* and *FWHM*,  influence its shape.

In [ ]:
xx = np.linspace(-10,10,100)

def interactive_plot(scale, center, hwhm):
    plt.plot(xx, lorentzian(xx, float(scale), float(center), float(hwhm)))
    plt.ylabel('lorentzian(x,{scale},{center},{hwhm})'.
               format(scale=scale, center=center, hwhm=hwhm))
    plt.xlabel('x')
    plt.grid()
    plt.show()

interactive_plot = ipywidgets.interactive(interactive_plot, 
                                          scale=(1, 10), 
                                          center=(0, 10), 
                                          hwhm=(1, 5))

def reset_values(b):
    """Reset the interactive plots to inital values."""
    interactive_plot.children[0].value = 5
    interactive_plot.children[1].value = 5
    interactive_plot.children[2].value = 3

reset_button = ipywidgets.Button(description = "Reset")
reset_button.on_click(reset_values)

output = interactive_plot.children[-1]
output.layout.height = '270px'
display(interactive_plot, reset_button)

(<a href='#Table of Contents'>Top</a>)<a id='ref_data'></a><h2>Creating reference data</h2>

**Input:** the reference data for this simple example correspond to a Lorentzian with added noise.


The fit is performed using `scipy.optimize.curve_fit`. <br> The example is based on implementations from https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

In [ ]:
# Creation of reference data
lorentzian_noisy = lorentzian(xx, 0.89, -0.025, 0.45)*(1. + 0.1*np.random.normal(0,1,100)) + 0.01*np.random.normal(0,1,100)

plt.plot(xx, lorentzian_noisy, label='reference data')
plt.xlabel('x')
plt.grid()
plt.legend()
plt.show()

(<a href='#Table of Contents'>Top</a>)<a id='fitting'></a><h2>Setting and fitting</h2>

In [ ]:
# From https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
# perform fit with initial guesses scale=1, center=0.2 HWHM=0.5
initial_parameters_values = [1, 0.2, 0.5]

plt.plot(xx, lorentzian_noisy, 'b-', label='reference data')
plt.plot(xx, lorentzian(xx, *initial_parameters_values), 'r.', label='model with initial guesses')
plt.xlabel('x')
plt.grid()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

popt, pcov = curve_fit(lorentzian, xx, lorentzian_noisy, p0=initial_parameters_values)

(<a href='#Table of Contents'>Top</a>)<a id='plot'></a><h2>Plotting the results</h2>

In [ ]:
# Calculation of the errors on the refined parameters:
perr = np.sqrt(np.diag(pcov))

print('Values of refined parameters:')
print('scale:', popt[0],'+/-', perr[0])
print('center :', popt[1],'+/-', perr[1])
print('HWHM', popt[2],'+/-', perr[2])

In [ ]:
# Comparison of reference data with fitting result
plt.plot(xx, lorentzian_noisy, 'b-', label='reference data')
plt.plot(xx, lorentzian(xx, *popt), 'g--', label='fit: %5.3f, %5.3f, %5.3f' % tuple(popt))
plt.legend()
plt.xlabel('x')
plt.grid()
plt.show()